In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df_eicu = pd.read_csv('eicu/eicu-gosiss-data.csv',header=0,sep=',')
df_anzics = pd.read_csv('anzics/anzics-gosiss-data.csv',header=0,sep=',')
df_mimic = pd.read_csv('mimic-iii/mimic-iii-gosiss-data.csv',header=0,sep=',')
df_orchestra = pd.read_csv('orchestra/orchestra-gosiss-data.csv',header=0,sep=',')

In [ ]:
# load in the header columns
hdr = pd.read_csv('hdr/header.csv',header=None,sep=',')[0].values

In [ ]:
# loop through and print whether the data is present
print('Is the data available for the given dataset?')
print('A blank indicates it is available. N/A specifies not available.')
print('')

print('{:30s}'.format('dataset'), end='\t')
data_source = ['anzics','eicu','mCV','mMV','orchestra']
print('\t'.join(data_source))

print_list = ['','','','','']
idxCV = df_mimic['data_source']=='carevue'
df_list = [df_anzics, df_eicu, df_mimic[idxCV], df_mimic[~idxCV], df_orchestra]
for c in hdr:
    print('{:30s}'.format(c),end="\t")
    
    for i, d in enumerate(df_list):
        if np.all(d[c].isnull()):
            print_list[i] = 'N/A'
        else:
            print_list[i] = ''
        
    print('\t'.join(print_list))

In [ ]:
# print the same as above to a file - with descriptions and categories
# load yaml definitions
with open("hdr/variable-definitions.yaml", 'r') as stream:
    try:
        varlist = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# convert to dataframe
df_var = pd.DataFrame.from_dict(varlist, orient='index')
df_var['varname'] = df_var.index

# specify the order of the categories - data is output in this order
category_order = {'identifier': 1,
                  'demographic': 2,
                  'APACHE covariate': 3,
                  'vitals': 4,
                  'labs': 5,
                  'labs blood gas': 6,
                  'APACHE prediction': 10}
df_var['category_order'] = df_var['category'].map(category_order)


# sort df by the category, then by the variable name
df_var.sort_values(['category_order','varname'],inplace=True)

data_source = ['anzics','eicu','mimic','orchestra']
df_list = [df_anzics, df_eicu, df_mimic, df_orchestra]
print_list = ['','','','','']

with open("GOSSIS_VARIABLE_COMPLETION.csv","w") as fp:
    fp.write('variables,category,description,')
    fp.write(','.join(data_source))
    fp.write('\n')
    
    for c in df_var.index:
        for i, d in enumerate(df_list):
            if np.all(d[c].isnull()):
                print_list[i] = 'N/A'
            else:
                print_list[i] = ''
        fp.write(c + ',')
        
        # write the category/description of the column
        fp.write('"' + df_var.loc[c, 'category'] + '",')
        fp.write('"' + df_var.loc[c, 'description'] + '",')
        
        # write whether data is available
        fp.write(','.join(print_list))
        fp.write('\n')
        

In [ ]:
# merge and spit out the data
df = pd.concat([df_eicu, df_anzics, df_mimic, df_orchestra],ignore_index=True)
df['data_source'].value_counts()

In [ ]:
# for the purposes of data comparison, combine carevue/metavision/both
df.loc[np.in1d(df['data_source'],['carevue','metavision','both']),'data_source'] = 'mimic'
df['data_source'].value_counts()

In [ ]:
# load the three dataframes
df.to_csv('gossis-data.csv',index=False)